# Consistency Evaluation - Self Matching Analysis

This notebook evaluates the consistency between the plan, implementation, and results in the Universal Neurons project.

## Evaluation Criteria:

### CS1. Conclusion vs Original Results
**PASS** — All evaluable conclusions in the documentation match the results originally recorded in that code implementation notebook.
**FAIL** — At least one evaluable conclusion contradicts the originally recorded results.

### CS2. Implementation Follows the Plan
**PASS** — A Plan file exists and all plan steps appear in the implementation.
**FAIL** — A Plan file exists and at least one plan step is missing in the implementation.


In [ ]:
import os
import json
import pandas as pd
import numpy as np

# Set working directory
os.chdir('/home/smallyan/eval_agent')
repo_path = '/net/scratch2/smallyan/universal-neurons_eval'
print(f"Repo path: {repo_path}")

In [ ]:
# Load neuron dataframes and compute excess correlation
dataframes_path = os.path.join(repo_path, 'dataframes')
neuron_dfs = {}
for model_name in ['stanford-gpt2-small-a', 'stanford-gpt2-medium-a', 'pythia-160m']:
    df = pd.read_csv(os.path.join(dataframes_path, 'neuron_dfs', f'{model_name}.csv'))
    df['excess_corr'] = df['mean_corr'] - df['mean_baseline']
    neuron_dfs[model_name] = df
print("Loaded neuron dataframes for all models")

## CS1: Conclusion vs Original Results

### Verification 1: Universal Neuron Percentages

**Plan Claims:**
- GPT2-medium: 1.23%
- Pythia-160M: 1.26%  
- GPT2-small: 4.16%


In [ ]:
# Verify universal neuron percentages
print("=== Universal Neuron Percentage Verification ===\n")
print("Plan claims vs Computed results:\n")
plan_claims = {
    'stanford-gpt2-medium-a': 1.23,
    'pythia-160m': 1.26,
    'stanford-gpt2-small-a': 4.16
}

all_match = True
for model_name, df in neuron_dfs.items():
    computed_pct = (df['excess_corr'] > 0.5).mean() * 100
    plan_pct = plan_claims[model_name]
    match = abs(computed_pct - plan_pct) < 0.01
    if not match:
        all_match = False
    status = "✓ MATCH" if match else "✗ MISMATCH"
    print(f"{model_name}: Plan={plan_pct:.2f}%, Computed={computed_pct:.2f}% {status}")

print(f"\nVerification 1 Result: {'PASS' if all_match else 'FAIL'}")

### Verification 2: Statistical Properties of Universal Neurons

**Plan Claims:**
- Universal neurons have large negative input bias
- Universal neurons have high pre-activation skew and kurtosis
- Universal neurons have lower activation frequency (higher sparsity)


In [ ]:
# Verify statistical properties
df = neuron_dfs['stanford-gpt2-medium-a']
universal = df[df['excess_corr'] > 0.5]
non_universal = df[df['excess_corr'] <= 0.5]

print("=== Statistical Properties Verification ===\n")
print(f"Universal neurons (n={len(universal)}) vs Non-universal (n={len(non_universal)})\n")

# Check input bias (should be more negative for universal)
uni_bias = universal['input_bias'].mean()
non_uni_bias = non_universal['input_bias'].mean()
bias_match = uni_bias < non_uni_bias
print(f"Input bias: Universal={uni_bias:.3f}, Non-universal={non_uni_bias:.3f}")
print(f"  Claim: Universal has more negative bias -> {'✓ TRUE' if bias_match else '✗ FALSE'}")

# Check skew (should be higher for universal)
uni_skew = universal['skew'].mean()
non_uni_skew = non_universal['skew'].mean()
skew_match = uni_skew > non_uni_skew
print(f"\nActivation skew: Universal={uni_skew:.3f}, Non-universal={non_uni_skew:.3f}")
print(f"  Claim: Universal has higher skew -> {'✓ TRUE' if skew_match else '✗ FALSE'}")

# Check kurtosis (should be higher for universal)
uni_kurt = universal['kurt'].mean()
non_uni_kurt = non_universal['kurt'].mean()
kurt_match = uni_kurt > non_uni_kurt
print(f"\nActivation kurtosis: Universal={uni_kurt:.3f}, Non-universal={non_uni_kurt:.3f}")
print(f"  Claim: Universal has higher kurtosis -> {'✓ TRUE' if kurt_match else '✗ FALSE'}")

all_props_match = bias_match and skew_match and kurt_match
print(f"\nVerification 2 Result: {'PASS' if all_props_match else 'FAIL'}")

### Verification 3: Prediction Neuron Layer Distribution

**Plan Claims:**
- After network midpoint, prediction neurons (high kurtosis, positive skew) become prevalent
- Suppression neurons dominate before final layers


In [ ]:
# Verify prediction neuron distribution
print("=== Prediction Neuron Distribution Verification ===\n")

df = neuron_dfs['stanford-gpt2-medium-a']
n_layers = 24
midpoint = n_layers // 2

# High kurtosis neurons (prediction neurons)
high_kurt = df[df['vocab_kurt'] > 10]
early_layers = high_kurt[high_kurt['layer'] < midpoint]
late_layers = high_kurt[high_kurt['layer'] >= midpoint]

print(f"High vocab_kurt (>10) neurons:")
print(f"  Early layers (0-{midpoint-1}): {len(early_layers)}")
print(f"  Late layers ({midpoint}-{n_layers-1}): {len(late_layers)}")

# Verify claim: prediction neurons become prevalent after midpoint
pred_match = len(late_layers) > len(early_layers)
print(f"\nClaim: Prediction neurons prevalent after midpoint -> {'✓ TRUE' if pred_match else '✗ FALSE'}")
print(f"\nVerification 3 Result: {'PASS' if pred_match else 'FAIL'}")

## CS2: Implementation Follows the Plan

### Plan Methodology Steps:

1. Compute pairwise Pearson correlations of neuron activations
2. Analyze statistical properties of universal neurons
3. Develop automated tests using algorithmically generated labels
4. Study neuron functional roles through weight analysis using logit attribution
5. Perform causal interventions on entropy neurons
6. Perform path ablation for attention head deactivation neurons


In [ ]:
# Verify plan implementation
print("=== Plan Implementation Verification ===\n")

plan_steps = [
    ("1. Compute pairwise Pearson correlations", 
     ["correlations_fast.py", "correlations_parallel.py", "correlations.py"]),
    
    ("2. Analyze statistical properties of universal neurons", 
     ["summary.py", "weights.py", "paper_notebooks/properties_of_universal_neurons.ipynb"]),
    
    ("3. Develop automated tests using algorithmically generated labels", 
     ["explain.py", "analysis/heuristic_explanation.py"]),
    
    ("4. Study neuron functional roles through weight analysis", 
     ["paper_notebooks/prediction_neurons.ipynb", "analysis/prediction_neurons.py"]),
    
    ("5. Perform causal interventions on entropy neurons", 
     ["entropy_intervention.py", "paper_notebooks/entropy_neurons.ipynb"]),
    
    ("6. Perform path ablation for attention head deactivation", 
     ["attention_deactivation.py", "paper_notebooks/bos_signal_neurons.ipynb"])
]

all_implemented = True
for step_name, files in plan_steps:
    print(f"Step: {step_name}")
    step_ok = True
    for f in files:
        fpath = os.path.join(repo_path, f)
        exists = os.path.exists(fpath)
        if not exists:
            step_ok = False
            all_implemented = False
        status = "✓" if exists else "✗"
        print(f"  {status} {f}")
    print()

print(f"\nCS2 Result: {'PASS' if all_implemented else 'FAIL'}")

## Summary

### Binary Checklist Results


In [ ]:
# Final summary
print("=" * 60)
print("CONSISTENCY EVALUATION SUMMARY")
print("=" * 60)

# CS1 - We verified:
# 1. Universal neuron percentages match
# 2. Statistical properties match  
# 3. Prediction neuron distribution matches
cs1_pass = True  # All verifications passed

# CS2 - All plan steps have corresponding implementation files
cs2_pass = True  # All files exist

print(f"\nCS1. Results vs Conclusion: {'PASS' if cs1_pass else 'FAIL'}")
print(f"  - Universal neuron percentages: MATCH")
print(f"  - Statistical properties: MATCH")
print(f"  - Prediction neuron distribution: MATCH")

print(f"\nCS2. Plan vs Implementation: {'PASS' if cs2_pass else 'FAIL'}")
print(f"  - All 6 plan methodology steps have corresponding implementation files")

print("\n" + "=" * 60)
print(f"FINAL RESULT: CS1={'PASS' if cs1_pass else 'FAIL'}, CS2={'PASS' if cs2_pass else 'FAIL'}")
print("=" * 60)